In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from joblib import dump, load
import json

## Preprocessing

In [2]:
data=pd.read_csv("dataset.csv")

In [3]:
print(data.head())
print(data.shape)

            Disease   Symptom_1              Symptom_2              Symptom_3  \
0  Fungal infection     itching              skin_rash   nodal_skin_eruptions   
1  Fungal infection   skin_rash   nodal_skin_eruptions    dischromic _patches   
2  Fungal infection     itching   nodal_skin_eruptions    dischromic _patches   
3  Fungal infection     itching              skin_rash    dischromic _patches   
4  Fungal infection     itching              skin_rash   nodal_skin_eruptions   

              Symptom_4 Symptom_5 Symptom_6 Symptom_7 Symptom_8 Symptom_9  \
0   dischromic _patches       NaN       NaN       NaN       NaN       NaN   
1                   NaN       NaN       NaN       NaN       NaN       NaN   
2                   NaN       NaN       NaN       NaN       NaN       NaN   
3                   NaN       NaN       NaN       NaN       NaN       NaN   
4                   NaN       NaN       NaN       NaN       NaN       NaN   

  Symptom_10 Symptom_11 Symptom_12 Symptom_13 Symp

In [4]:
for col in data.columns: 
    data[col] = data[col].str.replace('_',' ')
data.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin rash,nodal skin eruptions,dischromic patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin rash,nodal skin eruptions,dischromic patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal skin eruptions,dischromic patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin rash,dischromic patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin rash,nodal skin eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data["list_of_symptoms"] = 0
for i in range(data.shape[0]):
    values = data.iloc[i].values
    values = values.tolist()
    data["list_of_symptoms"][i] = values[1:values.index(0)]

print(data.tail)
print(data.head())

/var/folders/0l/srjf8wrn45d3c527z89b0d9m0000gn/T/ipykernel_20182/4233933007.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["list_of_symptoms"][i] = values[1:values.index(0)]


<bound method NDFrame.tail of                                       Disease             Symptom_1  \
0                            Fungal infection               itching   
1                            Fungal infection             skin rash   
2                            Fungal infection               itching   
3                            Fungal infection               itching   
4                            Fungal infection               itching   
...                                       ...                   ...   
4915  (vertigo) Paroymsal  Positional Vertigo              vomiting   
4916                                     Acne             skin rash   
4917                  Urinary tract infection   burning micturition   
4918                                Psoriasis             skin rash   
4919                                 Impetigo             skin rash   

                  Symptom_2              Symptom_3                  Symptom_4  \
0                 skin rash   nodal 

In [6]:
contains_zero = data['list_of_symptoms'].apply(lambda x: 0 in x)

print(data[contains_zero])

Empty DataFrame
Columns: [Disease, Symptom_1, Symptom_2, Symptom_3, Symptom_4, Symptom_5, Symptom_6, Symptom_7, Symptom_8, Symptom_9, Symptom_10, Symptom_11, Symptom_12, Symptom_13, Symptom_14, Symptom_15, Symptom_16, Symptom_17, list_of_symptoms]
Index: []


In [7]:
all_symptoms = data[['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4',
       'Symptom_5', 'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9',
       'Symptom_10', 'Symptom_11', 'Symptom_12', 'Symptom_13', 'Symptom_14',
       'Symptom_15', 'Symptom_16', 'Symptom_17']].values.ravel()
print(all_symptoms)

symptoms=pd.unique(all_symptoms.tolist())
#remove nun value
symptoms = [i for i in symptoms if str(i) != "nan"]

print(symptoms)
print(len(symptoms))

one_hot_data = pd.DataFrame(columns = symptoms,index = data.index)
#adding the disease
one_hot_data['list_of_symptoms']=data['list_of_symptoms']

['itching' ' skin rash' ' nodal skin eruptions' ... nan nan nan]
['itching', ' skin rash', ' nodal skin eruptions', ' dischromic  patches', ' continuous sneezing', ' shivering', ' chills', ' watering from eyes', ' stomach pain', ' acidity', ' ulcers on tongue', ' vomiting', ' cough', ' chest pain', ' yellowish skin', ' nausea', ' loss of appetite', ' abdominal pain', ' yellowing of eyes', ' burning micturition', ' spotting  urination', ' passage of gases', ' internal itching', ' indigestion', ' muscle wasting', ' patches in throat', ' high fever', ' extra marital contacts', ' fatigue', ' weight loss', ' restlessness', ' lethargy', ' irregular sugar level', ' blurred and distorted vision', ' obesity', ' excessive hunger', ' increased appetite', ' polyuria', ' sunken eyes', ' dehydration', ' diarrhoea', ' breathlessness', ' family history', ' mucoid sputum', ' headache', ' dizziness', ' loss of balance', ' lack of concentration', ' stiff neck', ' depression', ' irritability', ' visual di

In [8]:
one_hot_data

,itching,skin rash,nodal skin eruptions,dischromic patches,continuous sneezing,shivering,chills,watering from eyes,stomach pain,acidity,...,foul smell of urine,continuous feel of urine,skin peeling,silver like dusting,small dents in nails,inflammatory nails,blister,red sore around nose,yellow crust ooze,list_of_symptoms
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[itching, skin rash, nodal skin eruptions, ..."
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[ skin rash, nodal skin eruptions, dischromi..."
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[itching, nodal skin eruptions, dischromic ..."
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[itching, skin rash, dischromic patches, na..."
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[itching, skin rash, nodal skin eruptions, n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[ vomiting, headache, nausea, spinning move..."
4916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[ skin rash, pus filled pimples, blackheads,..."
4917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[ burning micturition, bladder discomfort, f..."
4918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[ skin rash, joint pain, skin peeling, silv..."


In [9]:
# Fill the data
for i in one_hot_data:
    one_hot_data[i] = data.apply(lambda x:1 if i in x.list_of_symptoms else 0, axis=1)
one_hot_data

,itching,skin rash,nodal skin eruptions,dischromic patches,continuous sneezing,shivering,chills,watering from eyes,stomach pain,acidity,...,foul smell of urine,continuous feel of urine,skin peeling,silver like dusting,small dents in nails,inflammatory nails,blister,red sore around nose,yellow crust ooze,list_of_symptoms
0,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4916,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4917,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
4918,0,1,0,0,0,0,0,0,0,0,...,0,0,1,1,1,1,0,0,0,0


In [10]:
one_hot_data['Disease']=data['Disease']
one_hot_data=one_hot_data.drop('list_of_symptoms',axis=1)

/var/folders/0l/srjf8wrn45d3c527z89b0d9m0000gn/T/ipykernel_20182/2825808893.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  one_hot_data['Disease']=data['Disease']


In [11]:
one_hot_data.rename(columns=lambda x: x.strip(), inplace=True)

In [12]:
one_hot_data

,itching,skin rash,nodal skin eruptions,dischromic patches,continuous sneezing,shivering,chills,watering from eyes,stomach pain,acidity,...,foul smell of urine,continuous feel of urine,skin peeling,silver like dusting,small dents in nails,inflammatory nails,blister,red sore around nose,yellow crust ooze,Disease
0,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo
4916,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Acne
4917,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,Urinary tract infection
4918,0,1,0,0,0,0,0,0,0,0,...,0,0,1,1,1,1,0,0,0,Psoriasis


In [13]:
symptoms_set = set(one_hot_data.columns)
print(symptoms_set)

{'pus filled pimples', 'irregular sugar level', 'knee pain', 'continuous feel of urine', 'swelling of stomach', 'pain in anal region', 'extra marital contacts', 'irritability', 'small dents in nails', 'redness of eyes', 'watering from eyes', 'mucoid sputum', 'cough', 'vomiting', 'throat irritation', 'nausea', 'sweating', 'inflammatory nails', 'enlarged thyroid', 'increased appetite', 'stomach pain', 'weight gain', 'runny nose', 'itching', 'loss of appetite', 'yellowish skin', 'slurred speech', 'spotting  urination', 'swelling joints', 'anxiety', 'swollen blood vessels', 'excessive hunger', 'weakness in limbs', 'swelled lymph nodes', 'acidity', 'patches in throat', 'red sore around nose', 'constipation', 'history of alcohol consumption', 'restlessness', 'weakness of one body side', 'yellowing of eyes', 'irritation in anus', 'blurred and distorted vision', 'lethargy', 'passage of gases', 'bruising', 'abnormal menstruation', 'pain behind the eyes', 'mood swings', 'ulcers on tongue', 'puff

## Training

In [14]:
train_data, test_data = train_test_split(one_hot_data, test_size=0.2, random_state=25)
X_train = train_data.drop("Disease",axis=1)
y_train = train_data["Disease"].copy()
X_test = test_data.drop("Disease",axis=1)
y_test = test_data["Disease"].copy()

In [15]:
RF_model = RandomForestClassifier()
RF_model.fit(X_train,y_train)

RandomForestClassifier()

In [16]:
print(X_train.shape)
print(y_train.shape)
y_pred=RF_model.predict(X_test)

(3936, 131)
(3936,)


In [17]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(cross_val_score(RF_model,X_train,y_train,cv=10).mean())

Accuracy: 1.0
1.0


### Save all symptoms in a text file

In [20]:
new_df = pd.DataFrame(X_test.iloc[302]).transpose()

print(new_df)

      itching  skin rash  nodal skin eruptions  dischromic  patches  \
1008        0          0                     0                    0   

      continuous sneezing  shivering  chills  watering from eyes  \
1008                    0          0       1                   0   

      stomach pain  acidity  ...  bladder discomfort  foul smell of urine  \
1008             0        0  ...                   0                    0   

      continuous feel of urine  skin peeling  silver like dusting  \
1008                         0             0                    0   

      small dents in nails  inflammatory nails  blister  red sore around nose  \
1008                     0                   0        0                     0   

      yellow crust ooze  
1008                  0  

[1 rows x 131 columns]


In [21]:
print(new_df.columns.tolist())

['itching', 'skin rash', 'nodal skin eruptions', 'dischromic  patches', 'continuous sneezing', 'shivering', 'chills', 'watering from eyes', 'stomach pain', 'acidity', 'ulcers on tongue', 'vomiting', 'cough', 'chest pain', 'yellowish skin', 'nausea', 'loss of appetite', 'abdominal pain', 'yellowing of eyes', 'burning micturition', 'spotting  urination', 'passage of gases', 'internal itching', 'indigestion', 'muscle wasting', 'patches in throat', 'high fever', 'extra marital contacts', 'fatigue', 'weight loss', 'restlessness', 'lethargy', 'irregular sugar level', 'blurred and distorted vision', 'obesity', 'excessive hunger', 'increased appetite', 'polyuria', 'sunken eyes', 'dehydration', 'diarrhoea', 'breathlessness', 'family history', 'mucoid sputum', 'headache', 'dizziness', 'loss of balance', 'lack of concentration', 'stiff neck', 'depression', 'irritability', 'visual disturbances', 'back pain', 'weakness in limbs', 'neck pain', 'weakness of one body side', 'altered sensorium', 'dark 

In [22]:
with open('symptoms.txt', 'w') as fp:
    for item in new_df.columns.tolist():
        fp.write("%s\n" % item)
    print('Done')

Done


In [24]:
predictions = RF_model.predict(new_df)
print(predictions)

['Typhoid']


## Exporting Modle

In [25]:
dump(RF_model, 'random_forest_model.joblib')

['random_forest_model.joblib']

In [26]:
loaded_rf_classifier = load('random_forest_model.joblib')

In [27]:
predictions = loaded_rf_classifier.predict(new_df)
print(predictions)

['Typhoid']


## Inferencing

In [2]:
symptoms_names = []

# open file and read the content in a list
with open('symptoms.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        symptoms_names.append(x)

# display list
print(symptoms_names)

['itching', 'skin rash', 'nodal skin eruptions', 'dischromic  patches', 'continuous sneezing', 'shivering', 'chills', 'watering from eyes', 'stomach pain', 'acidity', 'ulcers on tongue', 'vomiting', 'cough', 'chest pain', 'yellowish skin', 'nausea', 'loss of appetite', 'abdominal pain', 'yellowing of eyes', 'burning micturition', 'spotting  urination', 'passage of gases', 'internal itching', 'indigestion', 'muscle wasting', 'patches in throat', 'high fever', 'extra marital contacts', 'fatigue', 'weight loss', 'restlessness', 'lethargy', 'irregular sugar level', 'blurred and distorted vision', 'obesity', 'excessive hunger', 'increased appetite', 'polyuria', 'sunken eyes', 'dehydration', 'diarrhoea', 'breathlessness', 'family history', 'mucoid sputum', 'headache', 'dizziness', 'loss of balance', 'lack of concentration', 'stiff neck', 'depression', 'irritability', 'visual disturbances', 'back pain', 'weakness in limbs', 'neck pain', 'weakness of one body side', 'altered sensorium', 'dark 

In [3]:
loaded_rf_classifier = load('random_forest_model.joblib')

In [4]:
name =  '{ "data":["itching", "skin rash", "nodal skin eruptions", "dischromic  patches", "continuous sneezing"]}'
json_object = json.loads(name)
print(type(json_object))
print(type(json_object["data"]))

test_list = ['itching', 'skin rash', 'nodal skin eruptions', 'dischromic  patches', 'continuous sneezing']
df = pd.DataFrame(columns=symptoms_names)
print(df)
new_row = []
for item in symptoms_names:
    if any(item in items for items in test_list):
        new_row.append(1)
    else:
        new_row.append(0) 
df.loc[len(df)] = new_row

print(df)

<class 'dict'>
<class 'list'>
Empty DataFrame
Columns: [itching, skin rash, nodal skin eruptions, dischromic  patches, continuous sneezing, shivering, chills, watering from eyes, stomach pain, acidity, ulcers on tongue, vomiting, cough, chest pain, yellowish skin, nausea, loss of appetite, abdominal pain, yellowing of eyes, burning micturition, spotting  urination, passage of gases, internal itching, indigestion, muscle wasting, patches in throat, high fever, extra marital contacts, fatigue, weight loss, restlessness, lethargy, irregular sugar level, blurred and distorted vision, obesity, excessive hunger, increased appetite, polyuria, sunken eyes, dehydration, diarrhoea, breathlessness, family history, mucoid sputum, headache, dizziness, loss of balance, lack of concentration, stiff neck, depression, irritability, visual disturbances, back pain, weakness in limbs, neck pain, weakness of one body side, altered sensorium, dark urine, sweating, muscle pain, mild fever, swelled lymph node